<font color='orange' size=8><B>**LAB Assignment 3: Robot Simulation**</B></font>

<font size=5><B>Objective :</B></font>

1. ศึกษาการจำลองการเคลื่อนที่หุ่นยนต์ผ่านโปรแกรม Simulation (Gazebo)
2. ศึกษาการเชื่อมต่อ Sensor กับ ROS2
3. ศึกษาการสร้าง Algorithm สำหรับควบคุมตำแหน่งปลายมือของหุ่นยนต์


<center><img src="ros2_control.jpg" width="800"/></center>

<font size=5><B>Lab Instruction :</B></font>

นักศึกษาจะต้องออกแบบระบบควบคุมหุ่นยนต์ด้วย ROS2 โดยที่จะต้องเชื่อมต่อระหว่าง IMU กับ ROS2 ด้วย Lib 
XICRO จากนั้นจึงนำค่าที่ได้มาเป็นสัญญาณขาเข้าของ Algorithm X เพื่อควบคุมตำแหน่งปลายมือของหุ่นยนต์ใน
ระบบ Simulation มีขั้นตอนการทำและคะแนนดังต่อไปนี้ ( คะแนน 120/100 )

1. การเชื่อมต่อ IMU เข้ากับ Library XICRO (20%)
2. ทำโหนด Calibration ของ IMU (20%)     [ เป็นการคำนวณและเพิ่มค่าของ cov เข้าไปยัง message ]
3. Algorithm X สำหรับการสั่งงานหุ่นยนต์ (30%)
4. การนำไฟล์ URDF/XACRO ไปยังโปรแกรม GAZEBO (20%)
5. การติดตั้ง ROS2_CONTROLLER ไปยัง GAZEBO (20%)
6. การออกแบบโครงสร้าง Package ใน ROS2 (10%)

<font color='orange' size=6><B>ภาพรวมของหุ่นยนต์ COCOA V (3DOF)</B></font>

หุ่นยนต์ COCOA V ถูกออกแบบเป็นหุ่นยนต์ประเภท Articulated Robot ที่มี 3 Degree of Freedom โดยโปรแกรม Solidworks 2021 

<center><img src="img/CocoaV_01.png" width="600"/></center>
<center><img src="img/CocoaV_02.png" width="600"/></center>
<center><img src="img/CocoaV_03.png" width="600"/></center>



โดยเราออกแบบให้หุ่นยนต์ของพวกเราประกอบด้วย ข้อต่อแบบ Revolute ทั้งหมด 3 ข้อต่อและข้อต่อแบบ Fixed 2 ข้อต่อโดยจะสามารถจัดเรียงได้ดังนี้
        <ul>
        <li>world</li>
          <li>    "world_joint" type="fixed"</li>
        <li>link_0</li>
         <li>    "joint_rev_0_1" type="revolute"</li>
        <li>link_1</li>
         <li>    "joint_rev_1_2" type="revolute"</li>
        <li>link_2</li>
          <li>    "joint_rev_2_3" type="revolute"</li>
        <li>link_3</li>
         <li>    "joint_eff" type="fixed"</li>
        <li>end_effector</li>     
        </ul>  
        

<font color='white' size=3><B>การกำหนด coordinate system และ Center of Mass ให้แต่ละ link</B></font>
<center><img src="img/Coordinate_System_00.png" width="600"/></center>
<center><font color='Yellow' size=3><B>ภาพรวมของตำแหน่งของ Coordinate System และ Center of Mass</B></font></center>
<center><img src="img/Coordinate_System_01.png" width="600"/></center>
<center><font color='Yellow' size=3><B>Link_0</B></font></center>
<center><img src="img/Coordinate_System_02.png" width="600"/></center>
<center><font color='Yellow' size=3><B>Link_1</B></font></center>
<center><img src="img/Coordinate_System_03.png" width="600"/></center>
<center><font color='Yellow' size=3><B>Link_2</B></font></center>
<center><img src="img/Coordinate_System_04.png" width="600"/></center>
<center><font color='Yellow' size=3><B>Link_3</B></font></center>
<center><img src="img/Coordinate_System_05.png" width="600"/></center>
<center><font color='Yellow' size=3><B>End_effector</B></font></center>

<font color='white' size=3><B>การทำ Mass Property (kilograms * square meters)</B></font>

<center><img src="img/mass_01.png" width="600"/></center>
<center><font color='Yellow' size=3><B>Link_0</B></font></center>
<center><img src="img/mass_02.png" width="600"/></center>
<center><font color='Yellow' size=3><B>Link_1</B></font></center>
<center><img src="img/mass_03.png" width="600"/></center>
<center><font color='Yellow' size=3><B>Link_2</B></font></center>
<center><img src="img/mass_04.png" width="600"/></center>
<center><font color='Yellow' size=3><B>Link_3</B></font></center>
<center><img src="img/mass_05.png" width="600"/></center>
<center><font color='Yellow' size=3><B>End_effector</B></font></center>

<font color='white' size=3><B>การเขียนไฟล์ XACRO และ URDF.XACRO</B></font><br>

<font color='white' size=3>เลือกใช้ XACRO เนื่องจากสามารถเขียน XML ได้สั้นและอ่านง่าย การเขียนไฟล์ XACRO มีทั้งหมด 3 ไฟล์ ประกอบไปด้วย  </font>

1.cocoav_gazebo.xacro

2.cocoav_include.xacro

3.cocoav_robot.urdf.xacro

<font color='orange' size=5>cocoav_gazebo.xacro</font>
<ul>
<li>ในส่วนของหัวข้อ "ros2_control" จะเป็นการกำหนด Parameter ให้กับ joint แต่ละอัน</li>
<li>กำหนด parameter ให้กับ 3 command_interface ได้แก่ position, velocity, effort</li>
</ul>  

<img src="img/Gazebo_01.png" width=""/>

<ul>
<li>ในส่วนของหัวข้อ "gazebo reference" จะเป็นการกำหนด Parameter และ สีที่จะแสดงผลใน Gazebo ตามแต่ละ link</li>
</ul>  

<img src="img/Gazebo_02.png" width=""/>

<font color='orange' size=5>cocoav_include.xacro</font>
<ul>
<li>ไฟล์นี้จะเน้นไปที่การประกาศตัวแปรที่จะดึงไปใช้ใน cocoav_robot.urdf.xacro</li>
<li>โดยเราได้ทำการประกาศตัวแปรในส่วนของ Center of Mass</li>
</ul>  

<img src="img/cocoav_include_01.png" width=""/>


<font color='orange' size=5>cocoav_robot.urdf.xacro</font>
<ul>
<li>เป็นการกำหนด Parameter ให้กับ 3 หัวข้อ ที่จะนำไปใช้ในโปแกรมจำลอง ได้แก่ </li>

"visual" แสดงผลภาพใน Gazebo

"collision" กำหนดการชนให้กับ visual

"inertial" การกำหนดค่าความเฉื่อย
</ul>  

<img src="img/cocoav_robot_01.png" width=""/>

<ul>
<li>การกำหนด Joint</li>

"Joint Name" กำหนดชื่อของ Joint

"parent" กำหนด link แม่

"child" กำหนด link ลูก

"origin" กำหนดจุก origin

"axis" กำหนดแกนหมุน

"limit" กำหนด limit ของ joint

</ul>  
<img src="img/cocoav_robot_02.png" width=""/>
